In [3]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN"
!cp "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN/." . -a

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ConSinGAN		    LICENSE	   representative_selection.py
ConSinGAN_Generating.ipynb  main_train.py  requirements.txt
evaluate_model.py	    README.md	   selected_index_40.npz


In [2]:
! pip install cifar10_web
! pip install -U git+https://github.com/albumentations-team/albumentations

  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-65jy6_99
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-65jy6_99
     |████████████████████████████████| 634kB 4.7MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65168 sha256=9cc9d93e901f943baab0a3f8e4323d4167f680ca4d89293ad3762aa5d6d5d8b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mn2z1577/wheels/6f/77/82/86baf8aeda64a6de0f890cd0f2fb31acaf5545cc9c99ad21ba
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=33096100a297e25b01d21901ef1567bf4604c707e7998f933db26f6197c537e0
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installa

In [4]:
import main_train
import evaluate_model
from representative_selection import *
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [6]:
def generate_ConSinGAN_augmented_data(imbalanced_label = 3,
                                      drop_ratio = 0.4,
                                      num_representatives = 2,
                                      parent_path = "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN",
                                      augment_only = True,
                                      PCA__n_component = 50,
                                      DBSCAN__eps = 0.75,
                                      DBSCAN__min_sample = 3,
                                      ConSinGAN__lr_scale = 0.1,
                                      ConSinGAN__number_of_stage = 3
                                      ):
    os.chdir(parent_path)
    output, directory = pipeline_rep_selection(imbalanced_label = imbalanced_label,
                                               drop_ratio = drop_ratio,
                                               num_representatives = num_representatives,
                                               PCA__n_component = PCA__n_component,
                                               DBSCAN__eps = DBSCAN__eps,
                                               DBSCAN__min_sample = DBSCAN__min_sample)
    
    for i in range(num_representatives):
        main_train.ConSinGAN_train(os.path.join(directory, f"rep_selected/rep_{i}.png"),
                                  ConSinGAN__lr_scale, 
                                  ConSinGAN__number_of_stage,
                                  os.path.join(directory, f"model_saved/rep_{i}"))
    
    generate_sample_size = int(5000 * drop_ratio)
    generate_size_per_rep = int(np.ceil(generate_sample_size) / num_representatives)
    
    sample = np.vstack([evaluate_model.generate_image\
                        (os.path.join(directory, f"model_saved/rep_{i}"), generate_size_per_rep)\
                         for i in range(num_representatives)])
    sample = sample[np.random.choice(sample.shape[0], generate_sample_size, replace=False)]
    if augment_only: 
        X_train = sample
        y_train = imbalanced_label*np.ones(generate_sample_size).astype(int)
    else:
        X_train, y_train, X_test, y_test = get_cifar_10()
        X_train_imbalanced, y_train_imbalanced, X_deleted, y_deleted = get_imbalanced_dataset(X_train, y_train, label = imbalanced_label, drop_ratio= drop_ratio)
        X_train = np.vstack((X_train_imbalanced, sample))
        y_train = np.concatenate((y_train_imbalanced, imbalanced_label*np.ones(generate_sample_size).astype(int)))

        index = np.arange(X_train.shape[0])
        np.random.shuffle(index)
        X_train = X_train[index]
        y_train = y_train[index]
    np.savez(os.path.join(directory, f"label_{imbalanced_label}_drop_ratio_{drop_ratio}.npz"), X_train = X_train, y_train = y_train)
    return X_train, y_train

In [7]:
X_train, y_train = generate_ConSinGAN_augmented_data(imbalanced_label = 4,
                                                     drop_ratio = 0.4,
                                                     num_representatives = 2,
                                                     parent_path = "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN",
                                                     augment_only = True
                                                     PCA__n_component = 50,
                                                     DBSCAN__eps = 0.75,
                                                     DBSCAN__min_sample = 3,
                                                     ConSinGAN__lr_scale = 0.1,
                                                     ConSinGAN__number_of_stage = 3)

Downloaded cifar-10-binary.tar.gz to /root/data/cifar10


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth



Training model (/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN/label_4_drop_ratio_0.4_num_representatives_2_PCA__n_component_50_DBSCAN__eps_0.75_DBSCAN__min_sample_3/model_saved/rep_0)
Training model with the following parameters:
	 number of stages: 3
	 number of concurrently trained stages: 3
	 learning rate scaling: 0.1
	 non-linearity: lrelu
Training on image pyramid: [torch.Size([1, 3, 26, 26]), torch.Size([1, 3, 29, 29]), torch.Size([1, 3, 32, 32])]



stage [0/2]::   0%|          | 0/2000 [00:00<?, ?it/s]

Time for training: 254.66406869888306 seconds
Training model (/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN/label_4_drop_ratio_0.4_num_representatives_2_PCA__n_component_50_DBSCAN__eps_0.75_DBSCAN__min_sample_3/model_saved/rep_1)
Training model with the following parameters:
	 number of stages: 3
	 number of concurrently trained stages: 3
	 learning rate scaling: 0.1
	 non-linearity: lrelu
Training on image pyramid: [torch.Size([1, 3, 26, 26]), torch.Size([1, 3, 29, 29]), torch.Size([1, 3, 32, 32])]



stage [2/2]:: 100%|██████████| 2000/2000 [01:32<00:00, 21.55it/s]


Time for training: 249.8167598247528 seconds
Loading models...
Generating Samples...
Loading models...
Generating Samples...


Read-in

In [ ]:
dataset = np.load("/content/label_4_drop_ratio_0.4_num_representatives_2_PCA__n_component_50_DBSCAN__eps_0.75_DBSCAN__min_sample_3/label_4_drop_ratio_0.4,.npz")
dataset["X_train"].shape

(50000, 32, 32, 3)

In [ ]:
def generate_ConSinGAN_augmented_data_from_selected(imbalanced_label = 3,
                                                    drop_ratio = 0.4,
                                                    num_representatives = 2,
                                                    parent_path = "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN",
                                                    continue_training_from = 0,
                                                    augment_only = True,
                                                    PCA__n_component = 50,
                                                    DBSCAN__eps = 0.75,
                                                    DBSCAN__min_sample = 3,
                                                    ConSinGAN__lr_scale = 0.1,
                                                    ConSinGAN__number_of_stage = 3
                                                    ):
    os.chdir(parent_path)
    save_directory = generate_save_directory(imbalanced_label = imbalanced_label,
                                        drop_ratio = drop_ratio,
                                        num_representatives = num_representatives,
                                        PCA__n_component = PCA__n_component,
                                        DBSCAN__eps = DBSCAN__eps,
                                        DBSCAN__min_sample = DBSCAN__min_sample)
    directory = os.path.join(parent_path, save_directory)
    
    for i in range(continue_training_from, num_representatives):
        main_train.ConSinGAN_train(os.path.join(directory, f"rep_selected/rep_{i}.png"),
                                   ConSinGAN__lr_scale, 
                                   ConSinGAN__number_of_stage,
                                   os.path.join(directory, f"model_saved/rep_{i}"))
    
    generate_sample_size = int(5000 * drop_ratio)
    generate_size_per_rep = int(np.ceil(generate_sample_size) / num_representatives)
    
    sample = np.vstack([evaluate_model.generate_image\
                        (os.path.join(directory, f"model_saved/rep_{i}"), generate_size_per_rep)\
                         for i in range(num_representatives)])
    sample = sample[np.random.choice(sample.shape[0], generate_sample_size, replace=False)]

    if augment_only: 
        X_train = sample
        y_train = imbalanced_label*np.ones(generate_sample_size).astype(int)
    else:
        X_train, y_train, X_test, y_test = get_cifar_10()
        X_train_imbalanced, y_train_imbalanced, X_deleted, y_deleted = get_imbalanced_dataset(X_train, y_train, label = imbalanced_label, drop_ratio= drop_ratio)
        X_train = np.vstack((X_train_imbalanced, sample))
        y_train = np.concatenate((y_train_imbalanced, imbalanced_label*np.ones(generate_sample_size).astype(int)))

        index = np.arange(X_train.shape[0])
        np.random.shuffle(index)
        X_train = X_train[index]
        y_train = y_train[index]
    np.savez(os.path.join(directory, f"label_{imbalanced_label}_drop_ratio_{drop_ratio}.npz"), X_train = X_train, y_train = y_train)
    return X_train, y_train

In [1]:
X_train, y_train = generate_ConSinGAN_augmented_data_from_selected(imbalanced_label = 4,
                                                drop_ratio = 0.4,
                                                num_representatives = 2,
                                                parent_path = "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN",
                                                continue_training_from = 1,
                                                # which is the first rep you will train here
                                                # you need to delete the model generated before
                                                augment_only = True,
                                                PCA__n_component = 50,
                                                DBSCAN__eps = 0.75,
                                                DBSCAN__min_sample = 3,
                                                ConSinGAN__lr_scale = 0.1,
                                                ConSinGAN__number_of_stage = 3
                                                )

NameError: name 'generate_ConSinGAN_augmented_data_from_selected' is not defined

In [ ]:
def generate_ConSinGAN_augmented_data_from_trained(imbalanced_label = 3,
                                                  drop_ratio = 0.4,
                                                  num_representatives = 2,
                                                  parent_path = "/content/drive/My Drive/Columbia_Sept_19_/Class/COMS_4995_DEEP_LEARNING/ConSinGAN",
                                                  PCA__n_component = 50,
                                                  DBSCAN__eps = 0.75,
                                                  DBSCAN__min_sample = 3,
                                                  ConSinGAN__lr_scale = 0.1,
                                                  ConSinGAN__number_of_stage = 3
                                                  ):
    os.chdir(parent_path)
    directory = generate_save_directory(imbalanced_label = imbalanced_label,
                                        drop_ratio = drop_ratio,
                                        num_representatives = num_representatives,
                                        PCA__n_component = PCA__n_component,
                                        DBSCAN__eps = DBSCAN__eps,
                                        DBSCAN__min_sample = DBSCAN__min_sample)
    directory = os.path.join(parent_directory, save_directory)
    
    generate_sample_size = int(5000 * drop_ratio)
    generate_size_per_rep = int(np.ceil(generate_sample_size) / num_representatives)
    
    sample = np.vstack([evaluate_model.generate_image\
                        (os.path.join(directory, f"model_saved/rep_{i}"), generate_size_per_rep)\
                         for i in range(num_representatives)])
    sample = sample[np.random.choice(sample.shape[0], generate_sample_size, replace=False)]

    X_train, y_train, X_test, y_test = get_cifar_10()
    X_train_imbalanced, y_train_imbalanced, X_deleted, y_deleted = get_imbalanced_dataset(X_train, y_train, label = imbalanced_label, drop_ratio= drop_ratio)
    X_train = np.vstack((X_train_imbalanced, sample))
    y_train = np.concatenate((y_train_imbalanced, imbalanced_label*np.ones(generate_sample_size).astype(int)))

    index = np.arange(X_train.shape[0])
    np.random.shuffle(index)
    X_train = X_train[index]
    y_train = y_train[index]
    np.savez(os.path.join(directory, f"label_{imbalanced_label}_drop_ratio_{drop_ratio}.npz"), X_train = X_train, y_train = y_train)
    return X_train, y_train

In [ ]:
output, directory = pipeline_rep_selection(imbalanced_label = imbalanced_label,
                                               drop_ratio = drop_ratio,
                                               num_representatives = num_representatives,
                                               PCA__n_component = PCA__n_component,
                                               DBSCAN__eps = DBSCAN__eps,
                                               DBSCAN__min_sample = DBSCAN__min_sample)


In [ ]:
for i in range(output.shape[0]):
    main_train.ConSinGAN_train(os.path.join(directory, f"rep_selected/rep_{i}.png"),
                               ConSinGAN__lr_scale, 
                               ConSinGAN__number_of_stage,
                               os.path.join(directory, f"model_saved/rep_{i}"))

In [ ]:
sample = np.vstack([evaluate_model.generate_image(os.path.join(directory, f"model_saved/rep_{i}"), 10) for i in range(output.shape[0])])

In [ ]:
augmented = generate_ConSinGAN_augmented_data(imbalanced_label = 3,
                                              drop_ratio = 0.4,
                                              num_representatives = 2,
                                              generate_sample_size = 50,
                                              PCA__n_component = 50,
                                              DBSCAN__eps = 0.75,
                                              DBSCAN__min_sample = 3,
                                              ConSinGAN__lr_scale = 0.1,
                                              ConSinGAN__number_of_stage = 3)